## Inicializando Google Earth Engine

In [1]:
import ee # # Authenticates and initializes Earth Engine
import geemap # A dynamic module of Google Earth Engine
import fiona # For geopackage file handling 
import os #To set and modify filepaths
import json # For json objects

In [2]:
def ee_initialize(): ## Initializing in Google Earth Engine as function
    try:
        ee.Initialize()
    except:
        ee.Authenticate()
        ee.Initialize()
ee_initialize()

## Conviertiendo geopackage a geojson

In [5]:
def gpkg_to_geojson(in_gpkg, layer= None, out_json=None):
    """Converts specific layer from geopackage file to GeoJSON.
    Args:
        in_gpkg (str): File path or URL of geopackage file.
        layer (str, optional): Layer name or number of the geopackage file. Defaults first layer as None
        out_json (str, optional): File path of the output GeoJSON. Defaults to None.
    Returns:
        object: The json object representing the geopackage layer.
    """   
    ee_initialize()
    try: 
        import fiona
        import json  
        if os.path.exists(in_gpkg): # If the path is a filepath      
                path_or_bytes = os.path.abspath(in_gpkg)
                reader = fiona.open
                if out_json is None:  ## Obtaining empty output json
                    out_json = os.path.splitext(in_gpkg)[0] + ".json"
                elif os.path.exists(out_json): # If the out_json is duplicated
                    out_json = out_json.replace('.json', '_bk.json')  
                elif not os.path.exists(os.path.dirname(out_json)): # If the filepath has not been created yet
                    os.makedirs(os.path.dirname(out_json))    
        buffer=[]         
        with reader(path_or_bytes, layer = layer, enabled_drivers="GPKG") as features:
            for feature in features: #Reading each feature of geopackage we obtain a dict keys for json
                    ids = feature["id"]
                    atr = feature["properties"]
                    geom = feature["geometry"]
                    buffer.append(dict(type="Feature", id = ids, geometry=geom, properties=atr)) 

        with open(out_json, "w") as geojson: 
            geojson.write(json.dumps({"type": "FeatureCollection", #Writing in a json our buffer list
                                 "features":buffer}, indent=2))
            geojson.close()          
        with open(out_json) as f:
             json_data = json.load(f) #Reading a full json and return it as result
       
        return json_data

    except Exception as e:
        print(e)

In [6]:
def gpkg_to_ee(in_gpkg, layer=None):
    """Converts specific layer from geopackage file to  Earth Engine object.
    Args:
        in_gpkg (str): File path or URL of geopackage file.
        layer (str, optional): Layer name or number of the geopackage. Defaults first layer as None
        
    Returns:
        object: Earth Engine objects representing the geopackage layer.
    """
    ee_initialize()
    try:
        json_data = gpkg_to_geojson(in_gpkg, layer= layer) ## Converting geopackage file to geojson
        ee_object  = geemap.geojson_to_ee(json_data) ## Converting geojson to ee object
        return ee_object
    except Exception as e:
        print(e) 

##  Descargando coleccion de imagenes MOD13Q1

In [7]:
## Vector data from filepath
workspace = r"/home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR"
geegpkg = os.path.join(workspace,"Materiales/roi_area.gpkg")

## Creating a ee.FeatureCollection
area_ee= gpkg_to_ee(geegpkg)
area_ee

In [8]:
def scale_factor(image):
# scale factor for the MODIS MOD13Q1 product
    return image.multiply(0.0001).copyProperties(image, 
                                             ['system:time_start','system:time_end'])
def clipped(image):
    return image.clip(area_ee)

In [11]:
modis_dataset= ee.ImageCollection("MODIS/006/MOD13Q1").select('NDVI')\
      .filter(ee.Filter.date('2000-01-01', '2018-12-31'))\
      .map(clipped)\
      .map(scale_factor)  
ndvi = {
  "min": -1.0,
  "max": 1.0,
  "palette": [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}

Map= geemap.Map(center=[-13.48908,-72.36807],zoom=10)
Map.add_basemap('SATELLITE')
Map.addLayer(modis_dataset, ndvi, 'NDVI')
Map

Map(center=[-13.48908, -72.36807], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [17]:
out_dir = r"/home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados"
qa_dataset = ee.ImageCollection("MODIS/006/MOD13Q1").select("DetailedQA")\
      .filter(ee.Filter.date('2000-01-01', '2018-12-31'))\
      .map(clipped)

modis_dataset = ee.ImageCollection("MODIS/006/MOD13Q1").select("NDVI")\
      .filter(ee.Filter.date('2000-01-01', '2018-12-31'))\
      .map(clipped)\
      .map(scale_factor)  

count = int(qa_dataset.size().getInfo())
images_ndvi = modis_dataset.toList(count)
images_qa = qa_dataset.toList(count)  

for i in range(0, count):
    image_ndvi = ee.Image(images_ndvi.get(i))
    image_qa = ee.Image(images_qa.get(i))
    start = ee.Date(image_ndvi.get("system:time_start")).format('YYYY-MM-dd').getInfo() 
    end = ee.Date(image_ndvi.get("system:time_end")).format('YYYY-MM-dd').getInfo()
    filename_qa= "MOD13Q1_QA_250m_16_days" + "_" + "{}".format(start) +"_" +"{}".format(end) + ".tif" 
    filename_ndvi= "MOD13Q1_NDVI_250m_16_days" + "_" + "{}".format(start) +"_" +"{}".format(end) + ".tif" 
    geemap.ee_export_image(image_qa, filename= os.path.join(out_dir, "QA_MOD13Q1", filename_qa), scale=250,
                           region=area_ee.geometry(), crs="epsg:4326", file_per_band=False)
    geemap.ee_export_image(image_ndvi, filename= os.path.join(out_dir, "NDVI_MOD13Q1", filename_ndvi), scale=250,
                           region=area_ee.geometry(), crs="epsg:4326", file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2000-02-18_2000-03-05.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2000-03-05_2000-03-21.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2000-03-21_2000-04-06.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2000-04-06_2000-04-22.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13

Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2001-01-17_2001-02-02.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2001-02-02_2001-02-18.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2001-02-18_2001-03-06.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2001-03-06_2001-03-22.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2001-03-22_2001-

Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2002-01-01_2002-01-17.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2002-01-17_2002-02-02.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2002-02-02_2002-02-18.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2002-02-18_2002-03-06.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13

Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2002-12-03_2002-12-19.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2002-12-19_2003-01-04.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2003-01-01_2003-01-17.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2003-01-17_2003-02-02.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2003-02-02_2003-

Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2003-11-17_2003-12-03.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2003-12-03_2003-12-19.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2003-12-19_2004-01-04.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2004-01-01_2004-01-17.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13

...

Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2015-02-02_2015-02-18.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2015-02-18_2015-03-06.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2015-03-06_2015-03-22.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2015-03-22_2015-04-07.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13

Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2016-01-01_2016-01-17.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2016-01-17_2016-02-02.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2016-02-02_2016-02-18.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2016-02-18_2016-03-05.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2016-03-05_2016-

Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2016-12-18_2017-01-03.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2017-01-01_2017-01-17.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2017-01-17_2017-02-02.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2017-02-02_2017-02-18.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13

Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2017-11-17_2017-12-03.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2017-12-03_2017-12-19.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2017-12-19_2018-01-04.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2018-01-01_2018-01-17.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2018-01-17_2018-

Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2018-11-01_2018-11-17.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2018-11-17_2018-12-03.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2018-12-03_2018-12-19.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/RasterData/Resultados/QA_MOD13Q1/MOD13Q1_QA_250m_16_days_2018-12-19_2019-01-04.tif
